#Step19 변수 사용성 개선
## 19.1 변수 이름 지정

In [3]:
# 변수에 이름 붙여주기
import numpy as np

class Variable:
  '''
  중간 변수에 대해서는 미분값을 제거하는 모드를 추가
  '''
  def __init__(self, data, name=None):
    if data is not None:
      if not isinstance(data, np.ndarray):
        raise TypeError('{}은(는) 지원하지 않습니다.'.format(type(data)))

    self.data = data
    self.name = name # name 추가
    self.grad = None
    self.creator = None
    self.generation = 0 # 세대 수를 기록하는 변수 추가

  def set_creator(self, func):
    self.creator = func
    self.generation = func.generation + 1 # 세대를 기록(부모 세대 + 1)

  def backward(self, retain_grad=False):
    if self.grad is None:
      self.grad = np.ones_like(self.data)

    funcs = []
    seen_set = set()

    def add_func(f):
      if f not in seen_set:     # 같은 함수가 중복 추가 되는 것을 방지
                                # -> 두 elements가 같은 creator를 가지는 경우!
        funcs.append(f)
        seen_set.add(f)         
        funcs.sort(key=lambda x: x.generation)

    add_func(self.creator)
     
    while funcs :              
      f = funcs.pop()          
      gys = [output().grad for output in f.outputs] # output()(약한참조)의 grad 가져옴
      gxs = f.backward(*gys)                        # gxs를 구함
      if not isinstance(gxs, tuple):
        gxs = (gxs, )

      for x, gx in zip(f.inputs, gxs):            # 구한 역전파를 각 x.grad에 저장
        if x.grad is None:                        # 같은 변수일 경우에는 덮어쓰지 말고 더해주자!
          x.grad = gx
        else:
          x.grad = x.grad + gx

        if x.creator is not None:                 # 계속해서 역전파 수행
          add_func(x.creator)                     # x.creator 추가

      if not retain_grad: # retain_grad가 false 이면 중간 변수의 미분값을 모두 None으로 설정
        for y in f.outputs:
          y().grad = None # y는 약한 참조(weakref)

def as_array(x):
  if np.isscalar(x):
    # x가 np.float64 같은 scalar 타입인지 확인(일반 float도 확인됨)
    return np.array(x)
  return x

##19.2 ndarray 인스턴스 변수
Variable이 마치 데이터인것 처럼 보이게 하자! 왜냐? 사용자 입장에서 중요한것은 Variable안의 '데이터'이기 때문!

In [4]:
class Variable:
  '''
  중간 변수에 대해서는 미분값을 제거하는 모드를 추가
  '''
  def __init__(self, data, name=None):
    if data is not None:
      if not isinstance(data, np.ndarray):
        raise TypeError('{}은(는) 지원하지 않습니다.'.format(type(data)))

    self.data = data
    self.name = name # name 추가
    self.grad = None
    self.creator = None
    self.generation = 0 # 세대 수를 기록하는 변수 추가

  def set_creator(self, func):
    self.creator = func
    self.generation = func.generation + 1 # 세대를 기록(부모 세대 + 1)

  def backward(self, retain_grad=False):
    if self.grad is None:
      self.grad = np.ones_like(self.data)

    funcs = []
    seen_set = set()

    def add_func(f):
      if f not in seen_set:     # 같은 함수가 중복 추가 되는 것을 방지
                                # -> 두 elements가 같은 creator를 가지는 경우!
        funcs.append(f)
        seen_set.add(f)         
        funcs.sort(key=lambda x: x.generation)

    add_func(self.creator)
     
    while funcs :              
      f = funcs.pop()          
      gys = [output().grad for output in f.outputs] # output()(약한참조)의 grad 가져옴
      gxs = f.backward(*gys)                        # gxs를 구함
      if not isinstance(gxs, tuple):
        gxs = (gxs, )

      for x, gx in zip(f.inputs, gxs):            # 구한 역전파를 각 x.grad에 저장
        if x.grad is None:                        # 같은 변수일 경우에는 덮어쓰지 말고 더해주자!
          x.grad = gx
        else:
          x.grad = x.grad + gx

        if x.creator is not None:                 # 계속해서 역전파 수행
          add_func(x.creator)                     # x.creator 추가

      if not retain_grad: # retain_grad가 false 이면 중간 변수의 미분값을 모두 None으로 설정
        for y in f.outputs:
          y().grad = None # y는 약한 참조(weakref)

  # property 덕분에 메서드를 인스턴스 변수처럼 사용할 수 있음
  @property
  def shape(self):
    return self.data.shape


def as_array(x):
  if np.isscalar(x):
    # x가 np.float64 같은 scalar 타입인지 확인(일반 float도 확인됨)
    return np.array(x)
  return x

In [6]:
x = Variable(np.array([[1, 2, 3], [4, 5, 6]]))
print(x.shape) # x.shape() 대신 x.shape로 호출 가능!

(2, 3)


In [7]:
class Variable:
  '''
  중간 변수에 대해서는 미분값을 제거하는 모드를 추가
  '''
  def __init__(self, data, name=None):
    if data is not None:
      if not isinstance(data, np.ndarray):
        raise TypeError('{}은(는) 지원하지 않습니다.'.format(type(data)))

    self.data = data
    self.name = name # name 추가
    self.grad = None
    self.creator = None
    self.generation = 0 # 세대 수를 기록하는 변수 추가

  def set_creator(self, func):
    self.creator = func
    self.generation = func.generation + 1 # 세대를 기록(부모 세대 + 1)

  def backward(self, retain_grad=False):
    if self.grad is None:
      self.grad = np.ones_like(self.data)

    funcs = []
    seen_set = set()

    def add_func(f):
      if f not in seen_set:     # 같은 함수가 중복 추가 되는 것을 방지
                                # -> 두 elements가 같은 creator를 가지는 경우!
        funcs.append(f)
        seen_set.add(f)         
        funcs.sort(key=lambda x: x.generation)

    add_func(self.creator)
     
    while funcs :              
      f = funcs.pop()          
      gys = [output().grad for output in f.outputs] # output()(약한참조)의 grad 가져옴
      gxs = f.backward(*gys)                        # gxs를 구함
      if not isinstance(gxs, tuple):
        gxs = (gxs, )

      for x, gx in zip(f.inputs, gxs):            # 구한 역전파를 각 x.grad에 저장
        if x.grad is None:                        # 같은 변수일 경우에는 덮어쓰지 말고 더해주자!
          x.grad = gx
        else:
          x.grad = x.grad + gx

        if x.creator is not None:                 # 계속해서 역전파 수행
          add_func(x.creator)                     # x.creator 추가

      if not retain_grad: # retain_grad가 false 이면 중간 변수의 미분값을 모두 None으로 설정
        for y in f.outputs:
          y().grad = None # y는 약한 참조(weakref)

  # 다른 인스턴스 변수를 추가해 보자.
  # property 덕분에 메서드를 인스턴스 변수처럼 사용할 수 있음
  @property
  def shape(self):
    return self.data.shape

  @property
  def ndim(self):
    return self.data.ndim

  @property
  def size(self):
    return self.data.size

  @property
  def dtype(self):
    return self.data.dtype

## 19.3 len 함수와 print 함수
ndarray에서 len 함수 -> 첫번째 차원의 원소 

In [9]:
# len example
x = [1, 2, 3, 4]
print(len(x))

x = np.array([1, 2, 3, 4])
print(len(x))

x = np.array([[1, 2, 3], [4, 5, 6]])
print(len(x))

4
4
2


In [12]:
class Variable:
  '''
  중간 변수에 대해서는 미분값을 제거하는 모드를 추가
  '''
  def __init__(self, data, name=None):
    if data is not None:
      if not isinstance(data, np.ndarray):
        raise TypeError('{}은(는) 지원하지 않습니다.'.format(type(data)))

    self.data = data
    self.name = name # name 추가
    self.grad = None
    self.creator = None
    self.generation = 0 # 세대 수를 기록하는 변수 추가

  def set_creator(self, func):
    self.creator = func
    self.generation = func.generation + 1 # 세대를 기록(부모 세대 + 1)

  def backward(self, retain_grad=False):
    if self.grad is None:
      self.grad = np.ones_like(self.data)

    funcs = []
    seen_set = set()

    def add_func(f):
      if f not in seen_set:     # 같은 함수가 중복 추가 되는 것을 방지
                                # -> 두 elements가 같은 creator를 가지는 경우!
        funcs.append(f)
        seen_set.add(f)         
        funcs.sort(key=lambda x: x.generation)

    add_func(self.creator)
     
    while funcs :              
      f = funcs.pop()          
      gys = [output().grad for output in f.outputs] # output()(약한참조)의 grad 가져옴
      gxs = f.backward(*gys)                        # gxs를 구함
      if not isinstance(gxs, tuple):
        gxs = (gxs, )

      for x, gx in zip(f.inputs, gxs):            # 구한 역전파를 각 x.grad에 저장
        if x.grad is None:                        # 같은 변수일 경우에는 덮어쓰지 말고 더해주자!
          x.grad = gx
        else:
          x.grad = x.grad + gx

        if x.creator is not None:                 # 계속해서 역전파 수행
          add_func(x.creator)                     # x.creator 추가

      if not retain_grad: # retain_grad가 false 이면 중간 변수의 미분값을 모두 None으로 설정
        for y in f.outputs:
          y().grad = None # y는 약한 참조(weakref)

  # 다른 인스턴스 변수를 추가해 보자.
  # property 덕분에 메서드를 인스턴스 변수처럼 사용할 수 있음
  @property
  def shape(self):
    return self.data.shape

  @property
  def ndim(self):
    return self.data.ndim

  @property
  def size(self):
    return self.data.size

  @property
  def dtype(self):
    return self.data.dtype

  # __len__ 구현
  def __len__(self):
    return len(self.data)

In [13]:
x = Variable(np.array([[1, 2, 3], [4, 5, 6]]))
print(len(x)) # 굿 잘됨

2


In [15]:
class Variable:
  '''
  중간 변수에 대해서는 미분값을 제거하는 모드를 추가
  '''
  def __init__(self, data, name=None):
    if data is not None:
      if not isinstance(data, np.ndarray):
        raise TypeError('{}은(는) 지원하지 않습니다.'.format(type(data)))

    self.data = data
    self.name = name # name 추가
    self.grad = None
    self.creator = None
    self.generation = 0 # 세대 수를 기록하는 변수 추가

  def set_creator(self, func):
    self.creator = func
    self.generation = func.generation + 1 # 세대를 기록(부모 세대 + 1)

  def backward(self, retain_grad=False):
    if self.grad is None:
      self.grad = np.ones_like(self.data)

    funcs = []
    seen_set = set()

    def add_func(f):
      if f not in seen_set:     # 같은 함수가 중복 추가 되는 것을 방지
                                # -> 두 elements가 같은 creator를 가지는 경우!
        funcs.append(f)
        seen_set.add(f)         
        funcs.sort(key=lambda x: x.generation)

    add_func(self.creator)
     
    while funcs :              
      f = funcs.pop()          
      gys = [output().grad for output in f.outputs] # output()(약한참조)의 grad 가져옴
      gxs = f.backward(*gys)                        # gxs를 구함
      if not isinstance(gxs, tuple):
        gxs = (gxs, )

      for x, gx in zip(f.inputs, gxs):            # 구한 역전파를 각 x.grad에 저장
        if x.grad is None:                        # 같은 변수일 경우에는 덮어쓰지 말고 더해주자!
          x.grad = gx
        else:
          x.grad = x.grad + gx

        if x.creator is not None:                 # 계속해서 역전파 수행
          add_func(x.creator)                     # x.creator 추가

      if not retain_grad: # retain_grad가 false 이면 중간 변수의 미분값을 모두 None으로 설정
        for y in f.outputs:
          y().grad = None # y는 약한 참조(weakref)

  # 다른 인스턴스 변수를 추가해 보자.
  # property 덕분에 메서드를 인스턴스 변수처럼 사용할 수 있음
  @property
  def shape(self):
    return self.data.shape

  @property
  def ndim(self):
    return self.data.ndim

  @property
  def size(self):
    return self.data.size

  @property
  def dtype(self):
    return self.data.dtype

  # __len__ 구현
  def __len__(self):
    return len(self.data)

  # print 함수가 출력해주는 내용을 정의 -> __repr__메서드를 재정의하면됨
  def __repr__(self):
    if self.data is None:
      return 'variable(None)'

    p = str(self.data).replace('\n', '\n' + ' ' * 9)
    return 'variable(' + p + ')'

In [16]:
x = Variable(np.array([[1, 2, 3], [4, 5, 6]]))
print(x) # 굿 잘됨

variable([[1 2 3]
          [4 5 6]])
